In [11]:
# Input file paths
measured_time_vs_strain_data_file = "sample_input_csv_1.csv"
sensitivity_matrix_file = "strain_sensitivity_matrix.csv"


import numpy as np
import pandas as pd

def estimate_loads_from_strains(measured_time_vs_strain_data_file_path, sensitivity_matrix_file_path):
    # Load the CSV files using pandas to inspect the data and handle headers
    S_df = pd.read_csv(measured_time_vs_strain_data_file_path)
    A_df = pd.read_csv(sensitivity_matrix_file_path, header=None)

    # Convert the dataframes to numpy arrays, excluding the time column for S
    S_matrix = S_df.iloc[:, 1:].values  # Excludes the first column (Time [s])
    A_matrix = A_df.values  # Convert the strain sensitivity matrix to a numpy array

    # Perform the least squares estimate calculation for the load vector L_hat
    A_T = A_matrix.T
    A_T_A_inv = np.linalg.pinv(np.dot(A_T, A_matrix))  # Using pseudo-inverse
    A_T_A_inv_A_T = np.dot(A_T_A_inv, A_T)

    # Calculate the load vector estimate for all time steps
    L_hat_timeseries = np.dot(A_T_A_inv_A_T, S_matrix.T).T

    # Create a pandas dataframe for Time vs. Estimated loads
    time_column = S_df.iloc[:, 0].values  # First column is Time [s]
    estimated_loads_df = pd.DataFrame(L_hat_timeseries, columns=[f'Load {i+1}' for i in range(A_matrix.shape[1])])
    estimated_loads_df.insert(0, 'Time [s]', time_column)  # Insert the time column at the beginning

    # Save the dataframe to a CSV file
    output_csv_file_path = 'estimated_loads.csv'
    estimated_loads_df.to_csv(output_csv_file_path, index=False)

    return estimated_loads_df, output_csv_file_path


# Using the function to calculate the loads and get the dataframe and CSV file path
loads_df, csv_file_path = estimate_loads_from_strains(measured_time_vs_strain_data_file, sensitivity_matrix_file)
loads_df, csv_file_path




(   Time [s]  Load 1    Load 2
 0       0.1  -250.0  0.100000
 1       0.2  -500.0  0.200000
 2       0.3  -750.0  0.296667
 3       0.4 -1000.0  0.400000
 4       0.5 -1248.0  0.500000
 5       0.6 -1500.0  0.600000
 6       0.7 -1748.0  0.700000
 7       0.8 -2000.0  0.800000
 8       0.9 -2248.0  0.900000
 9       1.0 -2500.0  1.000000,
 'estimated_loads.csv')